In [261]:
import random
import pandas as pd
import os

In [262]:
def generate_random_text(bits=80):
    # calculate the number of characters needed to represent 80 bits
    chars = bits // 8
    # generate a random string of alphanumeric characters
    text = "".join(random.choice(string.ascii_letters + string.digits)
                   for _ in range(chars))
    # return the text
    return text

In [263]:
def convert_to_decimal_and_hexa(text):
    # encode the text as a bytes object using utf-8
    bytes = text.encode("utf-8")
    # convert the bytes object to an integer using int.from_bytes
    decimal_num = int.from_bytes(bytes, "big")
    hexa_num = hex(decimal_num)
    # return the number
    return decimal_num, hexa_num

In [264]:
def convert_to_text(number):
    # convert the number to a bytes object using int.to_bytes
    bytes = number.to_bytes((number.bit_length() + 7) // 8, "big")
    # decode the bytes object using utf-8
    text = bytes.decode("utf-8")
    # return the text
    return text

In [265]:
import string

# test the function
random_text = generate_random_text()
print(random_text)
decimal_num, hexa_num = convert_to_decimal_and_hexa(random_text)
print(hexa_num)
print(decimal_num)
string = convert_to_text(decimal_num)
print(string)

jd93egpN6c
0x6a6439336567704e3663
502419643341132262159971
jd93egpN6c


In [266]:
import string

# Specify the file path
file_path = 'output.csv'

# Clear the file content first
with open(file_path, "w") as file:
    file.write("")

# generate 100 random texts
for i in range(100):
    random_text = generate_random_text()
    # add each random text to a line in the file
    with open(file_path, "a") as file:
        file.write(random_text + "\n")

# remove the last newline character
with open(file_path, "rb+") as file:
    file.seek(-1, 2)
    file.truncate()

In [267]:
# Define the number of rows you want in your DataFrame
num_rows = 1000

# Generate random texts using your function
texts = [generate_random_text() for _ in range(num_rows)]

# Generate random odd degrees between 1 and 13 (inclusive)
degrees = [random.randrange(1, 14, 2) for _ in range(num_rows)]

# Create a DataFrame with columns "text" and "degree"
df = pd.DataFrame({"random_text": texts, "degree": degrees})

In [268]:
df['degree'].value_counts()

degree
5     154
7     149
13    144
1     143
3     142
9     135
11    133
Name: count, dtype: int64

In [269]:
def generate_data_points(degree):
    """
    Generates a list of random (x, y) data points based on the given degree.
    """
    data_points = []
    for _ in range(degree + 1):
        # Generate random x and y coordinates here.
        x = random.random()
        y = random.random()
        data_points.append((x, y))
    return data_points

In [270]:
df["data_points"] = df["degree"].apply(generate_data_points)

## Apply Lagrange to get the polynomial


In [271]:
from scipy.interpolate import lagrange
import numpy as np


def interpolate(data_points):
    """
    Returns a function that interpolates between the given data points.
    """
    # Extract the x and y coordinates from the data points
    x_coords, y_coords = zip(*data_points)
    # Apply lagrange interpolation to the data points
    polynomial = lagrange(x_coords, y_coords)
    # Return the polynomial function coefficients as a list
    return list(polynomial.coefficients)

In [272]:
# Try interpolating the first row of data points
x = interpolate(df["data_points"].iloc[0])

In [273]:
# Apply the interpolate function to the data points
df["coeffs"] = df["data_points"].apply(interpolate)

In [274]:
def nth_root(deg):
    if 1 <= deg < 10:
        return deg / 2
    elif 10 <= deg < 15:
        return deg / 3
    else:
        return deg

In [275]:
df["nth_root"] = df["degree"].apply(nth_root)

In [276]:
# Define a function to extract the constant term of a polynomial
def constant_term(polynomial):
    return polynomial[-1]

# Define a function to calculate the nth_root of a number


def nth_root(number, n):
    return np.sign(number) * np.abs(number) ** (1 / n)


# Create a new column called constant_nth_root by applying the functions to the polynomial and nth_root columns
df["constant_nth_root"] = df.apply(lambda row: nth_root(
    constant_term(row["coeffs"]), row["nth_root"]), axis=1)

In [277]:
# Import math library
import math


def bisection_initial(number):
    return [-round(number), round(number)]


def secant_initial(number):
    return [math.floor(number), math.ceil(number)]


def newton_initial(number):
    return round(number)


# Create a new column called bisection_initial by applying the function to the constant_nth_root column
df["bisection_initial"] = df["constant_nth_root"].apply(bisection_initial)
df["secant_initial"] = df["constant_nth_root"].apply(secant_initial)
df["newton_initial"] = df["constant_nth_root"].apply(newton_initial)

In [278]:
from scipy.optimize import bisect, newton


def bisection_method(coeffs, a, b, tol=1e-6, maxiter=200):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the bisection method using scipy's built-in function
    root = bisect(poly, a, b, xtol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)


def newton_method(coeffs, x0, tol=1e-6, maxiter=200):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the newton method using scipy's built-in function
    root = newton(poly, x0, tol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)


def secant_method(coeffs, x0, x1, tol=1e-6, maxiter=200):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the secant method using scipy's built-in function
    root = newton(poly, x0, x1=x1, tol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)

In [279]:
from scipy.optimize import bisect, newton


def bisection_method(coeffs, a, b, tol=1e-6, maxiter=200):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the bisection method using scipy's built-in function
    root = bisect(poly, a, b, xtol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)


def newton_method(coeffs, x0, tol=1e-6, maxiter=200):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the newton method using scipy's built-in function
    root = newton(poly, x0, tol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)


def secant_method(coeffs, x0, x1, tol=1e-6, maxiter=200):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the secant method using scipy's built-in function
    root = newton(poly, x0, x1=x1, tol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)

In [280]:
from scipy.optimize import bisect, newton

tolerance = 1e-4
maxiter = 1500


def bisection_method(coeffs, a, b, tol=tolerance, maxiter=maxiter):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the bisection method using scipy's built-in function
    root = bisect(poly, a, b, xtol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)


def newton_method(coeffs, x0, tol=tolerance, maxiter=maxiter):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the newton method using scipy's built-in function
    root = newton(poly, x0, tol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)


def secant_method(coeffs, x0, x1, tol=tolerance, maxiter=maxiter):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Apply the secant method using scipy's built-in function
    root = newton(poly, x0, x1=x1, tol=tol, maxiter=maxiter)
    # Return the root as a float
    return float(root)

In [ ]:
# # Create a new column called bisection_root by applying the function to the coeffs and bisection_initial columns
# df["bisection_root"] = df.apply(lambda row: bisection_method(
#     row["coeffs"], row["bisection_initial"][0], row["bisection_initial"][1]), axis=1)

# Create a new column called newton_root by applying the function to the coeffs and newton_initial columns
df["newton_root"] = df.apply(lambda row: newton_method(
    row["coeffs"], row["newton_initial"]), axis=1)

# Create a new column called secant_root by applying the function to the coeffs and secant_initial columns
df["secant_root"] = df.apply(lambda row: secant_method(
    row["coeffs"], row["secant_initial"][0], row["secant_initial"][1]), axis=1)

In [ ]:
# Define a function to evaluate a polynomial at a given root
def poly_val(coeffs, root):
    # Convert the list of coefficients to a numpy polynomial
    poly = np.poly1d(coeffs)
    # Evaluate the polynomial at the root
    val = poly(root)
    # Return the value as a float
    return float(val)

In [ ]:
# # Create a new column called poly_val_bisection_root by applying the function to the coeffs and bisection_root columns
# df["poly_val_bisection_root"] = df.apply(lambda row: poly_val(row["coeffs"], row["bisection_root"]), axis=1)

# Create a new column called poly_val_newton_root by applying the function to the coeffs and newton_root columns
df["poly_val_newton_root"] = df.apply(
    lambda row: poly_val(row["coeffs"], row["newton_root"]), axis=1)

# Create a new column called poly_val_secant_root by applying the function to the coeffs and secant_root columns
df["poly_val_secant_root"] = df.apply(
    lambda row: poly_val(row["coeffs"], row["secant_root"]), axis=1)

In [ ]:
df.to_csv(file_path, index=False, sep=';')